## **Ancient Footprints, Modern Pathways to Sustainability** - Part1: A comprehensive Upper Xingu Data Collection

## **Part 1: Data Integration and GPT-Guided Test Site Generation**

In the first phase of our analysis, we focus on building a **comprehensive geospatial dataset** that captures the environmental, climatic, and topographic complexity of the Upper Xingu landscape. This includes variables extracted from diverse sources:

- **LiDAR-derived elevation and slope** (via OpenTopography)  
- **Sentinel-2 satellite imagery** (NDVI, NDBI, BSI)  
- **Soil characteristics** from ISRIC SoilGrids (SOC, pH, clay, CEC, bulk density)  
- **19 bioclimatic variables** from WorldClim v2  
- **Distance to major river systems** (from GSHHG)

To our knowledge, this is the **first comprehensive integration** of such multimodal environmental features for archaeological modeling in the **Upper Xingu**, enabling high-resolution ecological characterization of both known and hypothetical sites.

A key novelty of this phase is the use of **synthetic test sites**, generated by prompting **GPT-4** with cultural-geographic logic, terrain patterns, and known site attributes. This allowed us to simulate 50 plausible but previously undocumented candidate coordinates—serving as a **test set for inference** in downstream classification. These GPT-generated points mirror how ancient Xinguano communities may have situated their settlements, offering a powerful and reproducible approach to exploratory archaeology in underexplored forest regions.


### LLM derived- Inference Site Coordinates

As part of our project goal to evaluate the likelihood of undiscovered archaeological sites in the Upper Xingu, we generated **50 synthetic test coordinates** to simulate novel candidates for model inference.

We began by extracting a subset of known Upper Xingu sites from the open-access dataset provided by [Walker et al. (2022)](https://www.science.org/doi/10.1126/sciadv.abj9727), which contains over 2,000 georeferenced archaeological locations across Amazonia.

To simulate plausible yet previously undocumented sites, we used **GPT-4** to generate 50 new coordinates based on known settlement logic and archaeological patterns. The model was primed with the following prompt:

> *"I will upload 30 data points from the Upper Xingu region, sourced from a known archaeological dataset. Using your prior archaeological knowledge, patterns from past discoveries, and geographic settlement logic, generate 50 new coordinate points within the Upper Xingu that are likely to represent undiscovered ADE or earthwork sites. You may base your reasoning on proximity to previously documented ADEs and earthworks, environmental factors, or known cultural clustering theories."*

This approach integrates **LLM-driven hypothesis generation** with environmental modeling, enabling us to explore where new archaeological discoveries might plausibly occur based on both data-driven prediction and cultural inference.


### LLM-Based Validation of GPT-Generated Test Sites

After generating 50 synthetic test site coordinates using GPT-4, we implemented a second layer of reasoning to validate their plausibility and novelty. The purpose of this step was twofold:

1. **Exclude locations with known archaeological documentation**, ensuring the test set only includes candidate sites that have not already been studied or published (e.g., via TerraBrasilis, research papers, or public databases).
2. **Filter out sites compromised by modern land-use change**, such as deforested zones, active agriculture, or urban development, using GPT-assisted cross-checking based on satellite observations and known deforestation records (e.g., [TerraBrasilis](http://terrabrasilis.dpi.inpe.br/)).

We prompted GPT-4 to reason through each of the 50 coordinates and return a qualitative assessment based on:
- Proximity to known archaeological zones (excluding overlap)
- Presence of intact forest or plausible pre-Columbian landscape
- Absence from existing documentation in public archives or known field surveys

This **LLM-guided screening process** ensured that our inference dataset consisted of **plausible, high-potential, and genuinely novel site candidates**—suitable for predictive modeling and future ground validation.


### River Distance Extraction

To capture hydrological proximity—a key factor in ancient settlement patterns—we calculated **Euclidean distance to major rivers** using the **Global Self-consistent, Hierarchical, High-resolution Geography Database (GSHHG v2.3.7)**.

We defined the Upper Xingu region with the following bounding box as our area of interest (AOI):
Longitude: -55.6175 to -52.7026
Latitude: -13.5525 to -11.0106

the following `ogr2ogr` code provides a reproducible example of how river features within this AOI can be extracted and reprojected to UTM Zone 22S:

```bash
ogr2ogr -f "ESRI Shapefile" \
  -t_srs EPSG:32722 \
  -spat -55.61752912 -13.55247573 -52.70255478 -11.01063107 \
  "/path/to/output/xingu_major1_utm.shp" \
  "$ROOT/WDBII_river_f_L03.shp"


### Climate Variable Extraction

To capture climatic variation across the Upper Xingu, we used 19 bioclimatic variables from **[WorldClim Version 2.1](https://www.worldclim.org/data/worldclim21.html)**, which provide long-term climate averages (1970–2000) at high spatial resolution (~1 km²).

We downloaded all **19 bioclimatic layers** (`.tif` format), which include variables such as:
- **Bio1**: Annual Mean Temperature  
- **Bio12**: Annual Precipitation  
- **Bio7**: Temperature Annual Range  
- **Bio19**: Precipitation of Coldest Quarter  
*(full list available on WorldClim website)*

Each layer was clipped to our AOI (Upper Xingu bounding box) and values were extracted at site coordinates using GIS tools or raster sampling in Python (e.g., `rasterio` or `earthpy`).



These **bioclimatic predictors** reflect key dimensions of temperature and precipitation that shaped **settlement suitability, agricultural potential**, and long-term landscape stability for Indigenous communities.

#### Example code to download the full bioclimatic dataset:

```bash
wget --content-disposition https://biogeo.ucdavis.edu/data/worldclim/v2.1/base/wc2.1_30s_bio.zip


### Setinel-2,Soil, Slope related Features 

Environmental Variables ([Google Earth Engine Extraction]([text](https://earthengine.google.com/))): Spectral, soil, and terrain variables were extracted using Google Earth Engine (GEE).  (n=2)
Sentinel-2 surface reflectance (COPERNICUS/S2_SR_HARMONIZED) imagery was filtered for cloud coverage (<20%) over the target region and composited using median reflectance across the observation period.  (n=3)
Soil variables were derived from ISRIC SoilGrids v2, specifically for the 15–30 cm depth, including pH (phh2o), organic carbon (SOC), clay content, cation exchange capacity (CEC), and bulk density (BDOD). Terrain features, including elevation and slope, were extracted using the NASA SRTM digital elevation model. (n=5)


In [1]:
import ee 
ee.Authenticate()
ee.Initialize()




Successfully saved authorization token.


In [ ]:
import pandas as pd 
df = pd.read_csv("/Users/hereagain/Desktop/OpenAItoZ/dataset/test_UpperXingu_start.csv")

In [9]:
df.columns

Index(['bulk_density', 'cec', 'clay', 'distriver1', 'distriver2', 'ph',
       'slope', 'soc', 'tri', 'type', 'longitude', 'latitude', 'bio1', 'bio2',
       'bio3', 'bio4', 'bio5', 'bio6', 'bio7', 'bio8', 'bio9', 'bio10',
       'bio11', 'bio12', 'bio13', 'bio14', 'bio15', 'bio16', 'bio17', 'bio18',
       'bio19', 'BSI', 'NDBI', 'NDVI'],
      dtype='object')

From the median composite, we computed three widely used indices:

- **NDVI (Normalized Difference Vegetation Index):** highlights green vegetation using bands B8 (NIR) and B4 (red)
- **NDBI (Normalized Difference Built-up Index):** emphasizes built-up or exposed features using B11 (SWIR) and B8 (NIR)
- **BSI (Bare Soil Index):** detects bare soil by combining bands B11, B4, B8, and B2

These indices were stacked into a multi-band raster and sampled at each site point, providing insight into **land cover characteristics** that may reflect anthropogenic disturbance or preserved archaeological surface patterns.

In [14]:

features = [
    ee.Feature(ee.Geometry.Point([row['longitude'], row['latitude']]), {'id': i})
    for i, row in df.iterrows()
]
points = ee.FeatureCollection(features)

### 1. Sentinel-2 Surface Indices
start = '2020-01-01'
end = '2021-12-31'

s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterDate(start, end) \
    .filterBounds(points.geometry()) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .median()

ndvi = s2.normalizedDifference(['B8', 'B4']).rename('NDVI')
ndbi = s2.normalizedDifference(['B11', 'B8']).rename('NDBI')

b11 = s2.select('B11')
b4 = s2.select('B4')
b8 = s2.select('B8')
b2 = s2.select('B2')
bsi = b11.add(b4).subtract(b8.add(b2)).divide(
    b11.add(b4).add(b8).add(b2)
).rename('BSI')

sentinel_stack = ndvi.addBands(ndbi).addBands(bsi)



In [ ]:

### 2. SoilGrids Features (15–30 cm depth)
ph = ee.Image("projects/soilgrids-isric/phh2o_mean").select("phh2o_15-30cm_mean").rename("ph")
soc = ee.Image("projects/soilgrids-isric/soc_mean").select("soc_15-30cm_mean").rename("soc")
clay = ee.Image("projects/soilgrids-isric/clay_mean").select("clay_15-30cm_mean").rename("clay")
cec = ee.Image("projects/soilgrids-isric/cec_mean").select("cec_15-30cm_mean").rename("cec")
bdod = ee.Image("projects/soilgrids-isric/bdod_mean").select("bdod_15-30cm_mean").rename("bulk_density")

soil_stack = ph.addBands(soc).addBands(clay).addBands(cec).addBands(bdod)

### 3. Terrain Features (Elevation, Slope, TRI)
elev = ee.Image("USGS/SRTMGL1_003").rename("elevation")
slope = ee.Terrain.slope(elev).rename("slope")

tri = elev.reduceNeighborhood(
    reducer=ee.Reducer.stdDev(),
    kernel=ee.Kernel.square(1)
).rename("tri")

terrain_stack = elev.addBands(slope).addBands(tri)

### 4. Combine all features
full_stack = sentinel_stack.addBands(soil_stack).addBands(terrain_stack)

### 5. Sample All at Once
sampled = full_stack.sampleRegions(
    collection=points,
    scale=30,
    geometries=True
)

# Convert to client-side list and keep original ID for rejoining
features_list = sampled.getInfo()['features']

records = []
for f in features_list:
    props = f['properties']
    coords = f['geometry']['coordinates']
    props['longitude'] = coords[0]
    props['latitude'] = coords[1]
    records.append(props)

# Create final DataFrame
df_features = pd.DataFrame(records).set_index('id')





In [23]:
df_features.head()

,BSI,NDBI,NDVI,bulk_density,cec,clay,elevation,id,ph,slope,soc,tri,longitude,latitude
0,0.277688,0.244562,0.265300,124,170,285,376,0,51,2.938511,196,1.065740,-54.054281,-13.239056
1,-0.281586,-0.306275,0.870549,127,64,292,339,1,48,1.327802,154,0.955814,-53.325028,-12.632424
2,0.112681,0.095268,0.525739,131,88,244,286,2,52,3.792345,157,1.227262,-53.266548,-12.352958
3,0.247000,0.225682,0.319289,131,78,207,329,3,49,0.927410,118,0.496904,-55.354053,-11.506745
4,0.230304,0.216391,0.368364,127,105,289,400,4,51,2.084417,165,0.993808,-53.964808,-13.214532


In [25]:
df_features = df_features.drop(columns=['longitude', 'latitude'], errors='ignore')

In [26]:
df_final = df.reset_index(drop=True).join(df_features.reset_index(drop=True), how='left')

In [28]:
df_final.head()

,longitude,latitude,distriver1,distriver2,bio1,bio2,bio3,bio4,bio5,bio6,...,NDVI,bulk_density,cec,clay,elevation,id,ph,slope,soc,tri
0,-54.054361,-13.238937,120202.560737,41888.893348,24.325000,25.033333,22.866667,1743.0,295.0,3.0,...,0.265300,124,170,285,376,0,51,2.938511,196,1.065740
1,-53.324932,-12.632489,222586.076474,73665.842083,24.629166,25.383333,23.150000,1674.0,290.0,1.0,...,0.870549,127,64,292,339,1,48,1.327802,154,0.955814
2,-53.266561,-12.352933,245487.103656,55371.915133,25.225000,26.000000,23.750000,1736.0,298.0,2.0,...,0.525739,131,88,244,286,2,52,3.792345,157,1.227262
3,-55.353967,-11.506791,21525.564974,164316.430523,25.358334,25.900000,24.650000,1879.0,318.0,1.0,...,0.319289,131,78,207,329,3,49,0.927410,118,0.496904
4,-53.964775,-13.214423,130162.573413,49880.052296,24.150000,24.933334,22.583334,1689.0,291.0,2.0,...,0.368364,127,105,289,400,4,51,2.084417,165,0.993808


### Open Topography 

### Feature Integration Summary

To construct a comprehensive dataset for both our **training sites** and **newly generated inference candidates** in the Upper Xingu, we integrated multiple layers of geospatial and environmental data from public sources. These variables capture ecological and anthropogenic signatures relevant to archaeological potential.

---

#### 🌊 River Distance
- **Source:** [GSHHG v2.3.7](https://www.soest.hawaii.edu/pwessel/gshhg/)
- **Method:** Extracted major river lines within the AOI (`-55.62, -13.55` to `-52.70, -11.01`) using `ogr2ogr` and projected to EPSG:32722 (UTM Zone 22S).

---

#### 🌦️ Climate (BIO1–BIO19)
- **Source:** [WorldClim v2.1](https://www.worldclim.org/data/worldclim21.html)
- Downloaded 19 global bioclimatic variables representing long-term averages of temperature and precipitation.

---

#### 🛰️ Sentinel-2 Indices
- **Source:** `COPERNICUS/S2_SR_HARMONIZED` via Google Earth Engine
- **Time Frame:** 2020–2021
- **Cloud Threshold:** < 20%
- **Extracted Indices:**
  - `NDVI`: Normalized Difference Vegetation Index  
  - `NDBI`: Normalized Difference Built-up Index  
  - `BSI`: Bare Soil Index

---

#### 🌱 Soil Characteristics
- **Source:** [SoilGrids (ISRIC)](https://soilgrids.org/)
- **Depth Layer:** 15–30 cm
- **Extracted Features:**
  - `ph`: Soil pH  
  - `soc`: Soil Organic Carbon  
  - `clay`: Clay content  
  - `cec`: Cation Exchange Capacity  
  - `bulk_density`: Bulk density

---

#### 🏔️ Topography
- **Source:** [USGS SRTMGL1_003 DEM](https://developers.google.com/earth-engine/datasets/catalog/USGS_SRTMGL1_003)
- **Derived Metrics:**
  - `elevation`  
  - `slope`  
  - `TRI`: Terrain Ruggedness Index (local standard deviation of elevation)

---

📍 All features were extracted using `sampleRegions()` in Google Earth Engine and merged back to the original coordinates of the candidate sites using index alignment to avoid precision mismatches in latitude/longitude.

> ✅ The same feature extraction process was applied to ~2,073 **known training sites** for consistent model input.
